# GPU / Computing Information

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Aug  9 16:51:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installing dependencies

In [1]:
# !pip install git+https://github.com/huggingface/transformers.git

In [2]:
# !pip uninstall tensorflow pytorch

# !pip install tensorflow
# !pip install pytorch

In [3]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install transformers==4.17.0
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install datasets==1.18.3


# Downloading and importing dataset

In [ ]:
from datasets import load_dataset, load_metric

training_dataset = load_dataset("addy88/sanskrit-asr-84", split='train[0:1%]')
testing_dataset = load_dataset("addy88/sanskrit-asr-84-eval", split='train[0:5%]')

Using custom data configuration addy88--sanskrit-asr-84-c9e988d1463fc8cc
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/addy88--sanskrit-asr-84-c9e988d1463fc8cc/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)
Using custom data configuration addy88--sanskrit-asr-84-eval-7d0a97e9b51bc11a
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/addy88--sanskrit-asr-84-eval-7d0a97e9b51bc11a/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


In [ ]:
print("training dataset --->\n\n", training_dataset)
print("\n\ntesting dataset --->\n\n", testing_dataset)

training dataset --->

 Dataset({
    features: ['audioFilename', 'snr', 'duration', 'gender', 'text', 'audio', '__index_level_0__'],
    num_rows: 372
})


testing dataset --->

 Dataset({
    features: ['audioFilename', 'snr', 'duration', 'gender', 'text', 'audio', '__index_level_0__'],
    num_rows: 465
})


# Preprocessing on the dataset

In [ ]:
training_dataset = training_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])
testing_dataset = testing_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])

print(len(training_dataset), len(testing_dataset))

372 465


In [ ]:
print("training dataset --->\n\n", training_dataset)
print("\n\ntesting dataset --->\n\n", testing_dataset)

training dataset --->

 Dataset({
    features: ['text', 'audio'],
    num_rows: 372
})


testing dataset --->

 Dataset({
    features: ['text', 'audio'],
    num_rows: 465
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(training_dataset.remove_columns(["audio"]), num_examples=10)

,text
0,राष्ट्रम् रामात् कुविन्दाः अद्य उत्तरप्रदेशे काम् पूर्वो नगर्योः यात्राम् निधास्यति
1,स्वस्य मन्त्री सूचयन् यत् साम्प्रतम् विदेशे यः समागतानाम् मेषम् अपि विमानरत्नम् चिकित्सके यम् परीक्षाम् त्रि विमानात् तेषु विधीयते
2,राष्ट्रम् सविशेषम् जगाद यत् संसदा विरते सत्रे
3,अत्रान्तरे भा कालेन निरतम् यत् उत्तरप्रदेशे लोक सभा निर्वचनम् सा पादयोः संयति निष्फला सञ्जाता
4,प्रधानम् त्रि नरेन्द्र नदी अद्य गजराजस्य दण्डी सूरयूः यात्राम् निधास्यति
5,स्वस्य मन्त्रान् स्पष्टा कृतम् यद् भारतम् तेषु रात्रिषु वर्तते सम्मिलितम् यत्र हि यदि यस्या गत्या सङ्क्रमणम् इदम् पराजयम् प्राप्नोति इति वार्ताः
6,केन्द्रम् सूचना प्रसरणम् त्रि प्रकाश जावळी करः स्पष्टरूपेण विषयम् आलक्ष्य मुदे प्रशंसनम् सर्वतो दिशा अवधानम् परम् वर्तते अपि च वार्तायाः नैके क्रमा अपि स मनुष्याः प्रासेन इति
7,वर्यस्य स्वप्नम् साक्षात् अभूत्
8,इण्निष्ठायाम् सहायो तेषु अष्टादशः इयाय खलु अद्य तृतीये दिन भारती क्रीडा भारतस्य पदम् तालिकाः संवर्धयन्ती इति आशास्य
9,पुनर् निर्मलस्य आधारशिलाः स्थापिता


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

# making huggingface datasets for vocab
vocab_train = training_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=training_dataset.column_names)
vocab_test = testing_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=testing_dataset.column_names)

# encoding the vocab in dictionary
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

Parameter 'function'=<function extract_all_chars at 0x7f9e1df71710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'ी': 0, 'ो': 1, 'म': 2, 'य': 3, 'ङ': 4, 'झ': 5, 'त': 6, 'घ': 7, 'श': 8, 'ओ': 9, 'प': 10, 'ब': 11, 'ै': 12, 'उ': 13, 'ध': 14, 'च': 15, '्': 16, 'ऐ': 17, 'ु': 18, 'ा': 19, 'ण': 20, 'ल': 21, ' ': 22, 'र': 23, 'आ': 24, 'स': 25, 'व': 26, 'क': 27, 'ड': 28, 'े': 29, 'थ': 30, 'ः': 31, 'ठ': 32, 'ृ': 33, 'ञ': 34, 'ख': 35, 'ि': 36, 'ए': 37, 'ं': 38, 'ळ': 39, 'इ': 40, 'अ': 41, 'ू': 42, 'ट': 43, 'औ': 44, 'ौ': 45, 'फ': 46, 'ग': 47, 'ह': 48, 'ढ': 49, 'भ': 50, 'द': 51, 'ष': 52, 'न': 53, 'ज': 54, 'ऽ': 55, 'छ': 56}


In [ ]:
# Adding additional vocab token

print("Initial vocab size :", len(vocab_dict))

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

print("Final vocab size :", len(vocab_dict))

Initial vocab size : 57
Final vocab size : 59


In [ ]:
# dumping the vocab in json file

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Starting huggingface transformers code

In [ ]:
# SETTING STUFF UP

# setting up Wav2Vec2 tokenizer

from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# setting up feature extractor

from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

# setting up Wav2Vec2 processor

from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/")

In [ ]:
# import torchaudio
import numpy as np

def speech_file_to_array_fn(batch):
    # speech_array, sampling_rate = torchaudio.load(batch["audio_path"])
    batch["speech"] = np.asarray(batch["audio"]["array"]) # speech_array[0].numpy()
    batch["sampling_rate"] = batch["audio"]["sampling_rate"]
    batch["target_text"] = batch["text"]
    return batch

training_dataset = training_dataset.map(speech_file_to_array_fn, remove_columns=training_dataset.column_names)
testing_dataset = testing_dataset.map(speech_file_to_array_fn, remove_columns=testing_dataset.column_names)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
print("training dataset --->\n\n", training_dataset)
print("\n\ntesting dataset --->\n\n", testing_dataset)

training dataset --->

 Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 372
})


testing dataset --->

 Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 465
})


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(training_dataset))

print(training_dataset[rand_int]["target_text"], "\n")
ipd.Audio(data=np.asarray(training_dataset[rand_int]["speech"]), rate=16000) #autoplay=True)

तेन करुणा सङ्क्रमणम् विरुध्य सङ्घर्षः अयम् निर्णयः कालः प्रतिपादिताः 



In [ ]:
rand_int = random.randint(0, len(training_dataset))

print("Target text:", training_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(training_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", training_dataset[rand_int]["sampling_rate"])

Target text: प्रवालो युवराजवर्य
Input array shape: (42240,)
Sampling rate: 16000


# More processing before training

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch



trainings_dataset = training_dataset.map(prepare_dataset, remove_columns=training_dataset.column_names, batch_size=8, num_proc=2, batched=True)
testings_dataset = testing_dataset.map(prepare_dataset, remove_columns=testing_dataset.column_names, batch_size=8, num_proc=2, batched=True)

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
print(trainings_dataset)
print(testings_dataset)

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 372
})
Dataset({
    features: ['input_values', 'labels'],
    num_rows: 465
})


In [ ]:
trainings_dataset = trainings_dataset.filter(lambda x: len(x) > 0, input_columns=["labels"])
testings_dataset = testings_dataset.filter(lambda x: len(x) > 0, input_columns=["labels"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(trainings_dataset)
print(testings_dataset)

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 371
})
Dataset({
    features: ['input_values', 'labels'],
    num_rows: 453
})


# Preparing for training and then doing training

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1712: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=1e-4,
  warmup_steps=500,
  save_total_limit=1,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=trainings_dataset,
    eval_dataset=testings_dataset,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


Usually training takes long... write the following code in the inspect of google colab to not allow the google colab server instance to disconnect

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
# Training

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 371
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 360


Step,Training Loss,Validation Loss,Wer
10,No log,20.999186,1.000000
20,No log,20.982073,1.000000
30,No log,20.943544,1.000000
40,No log,20.888485,1.000000


***** Running Evaluation *****
  Num examples = 453
  Batch size = 8
***** Running Evaluation *****
  Num examples = 453
  Batch size = 8
***** Running Evaluation *****
  Num examples = 453
  Batch size = 8
***** Running Evaluation *****
  Num examples = 453
  Batch size = 8


KeyboardInterrupt: ignored

# Testing

## Getting testing dataset

In [4]:
from datasets import load_dataset, load_metric
testing_dataset = load_dataset("addy88/sanskrit-asr-84-eval", split='train[0:5%]')

Using custom data configuration addy88--sanskrit-asr-84-eval-7d0a97e9b51bc11a


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/237M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/247M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/233M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/238M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/addy88--sanskrit-asr-84-eval-7d0a97e9b51bc11a/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


In [5]:
print("\n\ntesting dataset --->\n\n", testing_dataset)



testing dataset --->

 Dataset({
    features: ['audioFilename', 'snr', 'duration', 'gender', 'text', 'audio', '__index_level_0__'],
    num_rows: 465
})


## Preprocessing on testing dataset

In [6]:
testing_dataset = testing_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])
print(testing_dataset)

Dataset({
    features: ['text', 'audio'],
    num_rows: 465
})


In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(testing_dataset.remove_columns(["audio"]), num_examples=10)

,text
0,रक्त दुर्जय घटनायाः तेन विनिद्रो चारौ अपेक्षितः
1,योगात् वा जायमानान् मृतानाम्
2,दीर्घ हिंसा घटना तु त्रयः स्त्रियेति जनाः मिताः
3,
4,वाचि कामिता वेतालः
5,सत्ता जागरूकया सप्ताहे अद्य प्रभृति मन्यते यत् नवम्बर्मासस्य द्वितीयाम् यावत् चित्ते
6,राष्ट्रम् वक्ष्यामि करणेन शस्त्रास्त्रेति प्रकरणम् पूर्वस्य विधाय यान्तम् हितः निगृहीतः
7,विना देशे सोढुम् संयुक्ते वैशाखे समारुहत् समबोधयन् उपरिष्टात् यम् व्यङ्गा नु प्राग्
8,सम्प्रति वार्ताः श्रूयताम्
9,उत्तमा न्यायालये केन्द्रम् सास्य आधार योजना


In [9]:
# import torchaudio
import numpy as np

def speech_file_to_array_fn(batch):
    # speech_array, sampling_rate = torchaudio.load(batch["audio_path"])
    batch["speech"] = np.asarray(batch["audio"]["array"]) # speech_array[0].numpy()
    batch["sampling_rate"] = batch["audio"]["sampling_rate"]
    batch["target_text"] = batch["text"]
    return batch

testing_dataset = testing_dataset.map(speech_file_to_array_fn, remove_columns=testing_dataset.column_names)

Parameter 'function'=<function speech_file_to_array_fn at 0x7f963eed9050> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

In [10]:
print(testing_dataset)

Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 465
})


In [36]:
testing_dataset = testing_dataset.filter(lambda x: len(x) > 0, input_columns=["target_text"])
print(testing_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 453
})


## Getting Model

In [11]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

all_models = [
    "Harveenchadha/vakyansh-wav2vec2-sanskrit-sam-60",
    "addy88/wav2vec2-base-sanskrit-84h",
    "addy88/wav2vec2-sanskrit-stt",
    "KabiAI/wav2vec2-large-xls-r-300m-Sanskrit-Hindi",
    "Harveenchadha/wav2vec2-pretrained-clsril-23-10k",
    "frmccann/CLSRIL-23",
]

model_str = all_models[0]


processor = Wav2Vec2Processor.from_pretrained(model_str)
model = Wav2Vec2ForCTC.from_pretrained(model_str)

Downloading:   0%|          | 0.00/213 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

## Finding Wer

In [12]:
wer = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [25]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(element):

    global model, processor

    # pad input values and return pt tensor
    input_values = processor(element["speech"], sampling_rate=16_000, return_tensors="pt").input_values
    
    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)


    # transcribe
    # element["predicted"] = processor.decode(predicted_ids[0])
    element["predicted"] = processor.decode(predicted_ids[0], skip_special_tokens=True)

    return element

In [26]:
import torch

result = testing_dataset.map(evaluate, batched=False, remove_columns=testing_dataset.column_names)

0ex [00:00, ?ex/s]

ValueError: ignored

In [31]:
print("WER: {:2f}".format(100 * wer.compute(predictions=result["predicted"], references=result["target_text"])))

WER: 77.295285


In [35]:
show_random_elements(result, num_examples=20)

,target_text,predicted
0,शरीरिणः प्रधानम् त्रिकम् आश्वासयत् यदा वक्ति रूपेण प्रशस्ता च दुष्प्रवेशम् सद्यः उपमितम् सर्वम् भवायाम् विदन्ति,श्रीषेनः प्रधानमन्त्रिणम् आश्वासयात् यदासौ वैयक्तिकरूपेण प्रशासनद्वारा च दुष्प्रवादमेनम् सद्यः उपशमयितुम् सर्वसम्भवोपायान् विदधानोत्ते
1,मयि पञ्चदशदिनानि पर्यन्तम् प्रचलिष्यति,मयि पञ्चदश दिना्कपर्यन्तम् प्रचलिष्यति
2,वाचकाः निन्दन् कुमार निर्मलः,्रवाचका निरञ्जन्कुमार्निर्वलः
3,इन्द्रेण भारतम् षड्भिः क्रीडतः पराजितम्,इङ्लण्डेन भारतम् षड्भीक्रीडक इः पराजिताम्
4,तच्च श्रृङ्गस्य कर्तरि विदिशा मन्त्रिणः अपि दिवङ्गता नेत्रे दधानः समर्पितः,अत्च श्रृणङ्कादिशस्य कारिकारिविदिश्मन्त्रिणा अपि दिवङ्गताये नेत्रेषद्धानिरिः समर्पिता
5,स्वातन्त्र्य उत्तरकाले अयम् प्रथम सरः आसीत्,्वातन्त्र्योत्तरकाले अयम् प्रथमो वसरः आसीत्
6,पराक् पतिः शरीरम् व्यङ्गा डः नदीम् यत् महिमानम् स्वीकरणात् यस्य केन्द्रम् बालकानाम् शिक्षा भवेत्,उपराष्टपतिः श्रीअम् व्यङ्कया नऐडुः न्यगादीत् यत् महिलानाम् सवलीकरणाभ्यानस्य केन्द्रम् बालिकानाम् शिक्षा भवेत्
7,कुमारे,तूमवाचरे
8,किम् उक्तम् यत् जलौघ प्रभावेति कीलः सर्वम् भवम् साहाय्यम् प्रदास्यति,किन्द्रिणोक्तम् यत् जलौहप्रभावित् कीरलाय सर्वसम्भवम् साहय्यम् प्रदाश्यति
9,इति वार्ताः,इति वार्ताः


# Inference

In [44]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse

def parse_transcription(model, wav_file=None, audio_input=None, sample_rate=None):
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained(model)
    model = Wav2Vec2ForCTC.from_pretrained(model)

    # load audio
    if wav_file:
      audio_input, sample_rate = sf.read(wav_file)

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)

    return transcription

In [45]:
# All Models :-

# Harveenchadha/vakyansh-wav2vec2-sanskrit-sam-60
# addy88/wav2vec2-base-sanskrit-84h (ERROR IN LOADING)
# addy88/wav2vec2-sanskrit-stt
# KabiAI/wav2vec2-large-xls-r-300m-Sanskrit-Hindi (ERROR IN LOADING)
# Harveenchadha/wav2vec2-pretrained-clsril-23-10k
# frmccann/CLSRIL-23



audio_input = testing_dataset[0]["speech"]
sample_rate = 16000
model = "addy88/wav2vec2-sanskrit-stt"

print("Predicted ==>\n", parse_transcription(audio_input, sample_rate))
print("\nActual ==>\n", testing_dataset[0]["target_text"])

Predicted ==>
 नवशत्कोटि रुप्यकैः नगरस्य स्थापना भविष्यति

Actual ==>
 नवम् कोटी रक्तः नगरस्य स्थापनम् भविष्यति
